# Cas Kaggle ApC
## Participants
Martí Armengod Villar 

Joan Marc Samó Rojas

## Objectius

https://www.kaggle.com/datasets/datasnaek/chess


En la nostra Base de dades el que volem és ser capaços de predir quin jugador guanyara a partir de la seva llencçada inicial.

